In [4]:
import os
import sys

# On remonte d'un niveau vers la racine du projet
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: e:\antibody-seq-developability\antibody-seq-developability


In [5]:
import numpy as np
import pandas as pd

from src.data_oas import load_oas_human_paired
from src.features_cdr import compute_cdr3_features

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report


In [6]:
from datetime import date

def filter_reasonable_cdr3(
    df: pd.DataFrame,
    min_len: int = 5,
    max_len: int = 30,
) -> pd.DataFrame:
    """
    Filter out VH CDR3 sequences with extreme lengths.
    Keeps rows where len(vh_cdr3) is between min_len and max_len (inclusive).
    """
    df = df.copy()
    lengths = df["vh_cdr3"].str.len()
    return df[lengths.between(min_len, max_len)]


def build_devscore_dataset(
    n_sample: int = 200_000,
    random_state: int = 0,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load OAS, filter CDR3 lengths, subsample, compute CDR3 features,
    and create a toy 'devscore' + binary label dev_label.
    """
    # 1) Load + filter
    df = load_oas_human_paired()
    df = filter_reasonable_cdr3(df, min_len=5, max_len=30)

    # 2) Subsample for speed
    if n_sample is not None and len(df) > n_sample:
        df = df.sample(n_sample, random_state=random_state).reset_index(drop=True)

    # 3) Compute CDR3 physico-chemical features
    df_feat = compute_cdr3_features(df)

    # 4) Standardize core features and build devscore
    for col in ["cdr3_len", "cdr3_hydro_mean", "cdr3_charge"]:
        mean = df_feat[col].mean()
        std = df_feat[col].std()
        df_feat[col + "_z"] = (df_feat[col] - mean) / std

    df_feat["devscore"] = (
        df_feat["cdr3_len_z"]
        + df_feat["cdr3_hydro_mean_z"]
        + df_feat["cdr3_charge_z"].abs()
    )

    # 5) Binarize using quantiles (30/70)
    q_low, q_high = df_feat["devscore"].quantile([0.3, 0.7])
    df_bin = df_feat[
        (df_feat["devscore"] <= q_low) | (df_feat["devscore"] >= q_high)
    ].copy()
    df_bin["dev_label"] = (df_bin["devscore"] >= q_high).astype(int)

    return df_feat, df_bin


In [7]:
df_feat, df_bin = build_devscore_dataset(
    n_sample=200_000,
    random_state=0,
)

df_feat.head(), df_bin["dev_label"].value_counts(normalize=True)


Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

(      run_id                                             vh_seq    vh_cdr1  \
 0  1287176_1  EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...   GFTFSSYA   
 1   1a_S1__1  QVQLQESGPRLVKPSGTLSLTCAVSGGSISSSNWWSWVRQPPGKGL...  GGSISSSNW   
 2  1287185_1  QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLE...   GFTFSDYY   
 3  1287146_1  QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLE...   GGSISSYY   
 4  1287160_1  QVQLQQWGAGLLKPSETLSLTCAVYGGSFSGYYWSWIRQPPGKGLE...   GGSFSGYY   
 
     vh_cdr2                vh_cdr3  \
 0  ISGSGGST          AKLNYYDSSGSDY   
 1   IYHSGST  ARELKEIIGATRGYYYYGMDV   
 2  ISSSGSTI          AREDYGDLYYFDY   
 3   IYYSGST         ARPIAAAFDWYFDL   
 4   INHSGST             AHQAYGGFDY   
 
                                               vl_seq      vl_cdr1 vl_cdr2  \
 0  DIVMTQSPLSLPVTPGEPASISCRSSQSLLHSNGYNYLDWYLQKPG...  QSLLHSNGYNY     LGS   
 1  EIVLTQSPATLSLSPGERATLSCRASQSVSSNLAWYQQKPGQAPRL...       QSVSSN     DTS   
 2  QSALTQPRSVSGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAP... 

In [11]:
from sklearn.model_selection import train_test_split

feature_cols = ["cdr3_len", "cdr3_hydro_mean", "cdr3_charge", "cdr3_aromatic_frac"]

X = df_bin[feature_cols].values
y = df_bin["dev_label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()


((96009, 4), (24003, 4), 0.49997396077451073, 0.49997916927050784)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

logreg_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1_000)),
])

logreg_clf.fit(X_train, y_train)

y_proba_logreg = logreg_clf.predict_proba(X_test)[:, 1]
y_pred_logreg = logreg_clf.predict(X_test)

print("=== Logistic regression baseline ===")
print("ROC-AUC:", roc_auc_score(y_test, y_proba_logreg))
print(classification_report(y_test, y_pred_logreg))


LogReg ROC-AUC: 0.9984285040069357
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     12002
           1       1.00      0.98      0.99     12001

    accuracy                           0.99     24003
   macro avg       0.99      0.99      0.99     24003
weighted avg       0.99      0.99      0.99     24003



In [10]:
rf_clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
)

rf_clf.fit(X_train, y_train)

y_proba_rf = rf_clf.predict_proba(X_test)[:, 1]
y_pred_rf = rf_clf.predict(X_test)

print("RF ROC-AUC:", roc_auc_score(y_test, y_proba_rf))
print(classification_report(y_test, y_pred_rf))


RF ROC-AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12002
           1       1.00      1.00      1.00     12001

    accuracy                           1.00     24003
   macro avg       1.00      1.00      1.00     24003
weighted avg       1.00      1.00      1.00     24003

